In [7]:
from openai import OpenAI
from google.colab import userdata
import json
# Create an OpenAI client with your deepinfra token and endpoint
openai = OpenAI(
    api_key=userdata.get('deepinfra'),
    base_url="https://api.deepinfra.com/v1/openai",
)

from requests import get

def get_nhsta_complaints_by_vehicle(make: str, model: str, modelYear: str):
  """https://api.nhtsa.gov/complaints/complaintsByVehicle?make=acura&model=rdx&modelYear=2012"""
  base_url = "https://api.nhtsa.gov/complaints/complaintsByVehicle"
  payload = {
      "make": make,
      "model": model,
      "modelYear": modelYear
  }
  r = get(base_url, params=payload)
  return json.dumps(r.json())

In [8]:
# here is the definition of our function
tools = [
    {
      "type": "function",
      "function": {
          "name": "get_nhsta_complaints_by_vehicle",
          "description": "Get NHSTA Complaints by vehicle and model year",
          "parameters": {
              "type": "object",
              "properties": {
                  "make": {
                      "type": "string",
                      "description":
                          "The make of the vehicle, e.g. Acura"
                  },
                  "model": {
                      "type": "string",
                      "description": "The model of the vehicle, e.g. ILX"
                  },
                  "modelYear": {
                      "type": "string",
                      "description": "The model year of the vehicle, e.g. 2022",
                  }
              },
              "required": ["make", "model", "modelYear"]
          },
      }
    }
  ]

# here is the user request
messages = [
    {
        "role": "user",
        "content": "What's up with the 2024 Tesla Cybertruck?"
    }
]

# let's send the request and print the response
response = openai.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-70B-Instruct",
    messages=messages,
    tools=tools,
    tool_choice="auto",
)
tool_calls = response.choices[0].message.tool_calls
for tool_call in tool_calls:
    print(json.dumps(tool_call.model_dump(), indent=3))


{
   "id": "call_xI7WB2OF6J6I9R0w6Ex9eimA",
   "function": {
      "arguments": "{\"make\": \"Tesla\", \"model\": \"Cybertruck\", \"modelYear\": \"2024\"}",
      "name": "get_nhsta_complaints_by_vehicle"
   },
   "type": "function"
}


In [11]:
from pprint import pprint
pprint(response)

ChatCompletion(id='chatcmpl-01d25c69a6894faa9bbc746b5d8c716d', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_xI7WB2OF6J6I9R0w6Ex9eimA', function=Function(arguments='{"make": "Tesla", "model": "Cybertruck", "modelYear": "2024"}', name='get_nhsta_complaints_by_vehicle'), type='function')], name=None))], created=1735918705, model='meta-llama/Meta-Llama-3.1-70B-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=35, prompt_tokens=317, total_tokens=352, completion_tokens_details=None, prompt_tokens_details=None, estimated_cost=8.691e-05))


In [12]:
# extend conversation with assistant's reply
messages.append(response.choices[0].message)

for tool_call in tool_calls:
  function_name = tool_call.function.name
  if function_name == "get_nhsta_complaints_by_vehicle":  #"get_current_weather":
      function_args = json.loads(tool_call.function.arguments)
      function_response = get_nhsta_complaints_by_vehicle(
          make=function_args.get("make"),
          model=function_args.get("model"),
          modelYear=function_args.get("modelYear"),
      )

  # extend conversation with function response
  messages.append({
      "tool_call_id": tool_call.id,
      "role": "tool",
      "content": function_response,
  })


# get a new response from the model where it can see the function responses
second_response = openai.chat.completions.create(
  model="meta-llama/Meta-Llama-3.1-70B-Instruct",
  messages=messages,
  tools=tools,
  tool_choice="auto",
)

print(second_response.choices[0].message.content)


There are 27 complaints about the 2024 Tesla Cybertruck. Some of the issues reported include:

* Windshield wiper failure
* Steering issues
* Electrical system problems
* Suspension issues
* Fuel/propulsion system problems
* Rear trim piece missing clips
* Critical error messages
* Loss of system redundancy
* Power reduced and steering performance reduced due to overheating
* Critical steering issue detected
* Accelerator pedal slipped and lodged in trim making vehicle pin the acceleration pedal to the floor

It's worth noting that some of these issues may be related to the vehicle's design or manufacturing process, while others may be isolated incidents. However, it's clear that there are some significant problems with the 2024 Tesla Cybertruck that need to be addressed.
